In [1]:
pip install mediapipe

Note: you may need to restart the kernel to use updated packages.


In [13]:
import mediapipe as mp
import cv2
import numpy as np
import uuid
import math
import os
import time

In [15]:


mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands
# Lista para armazenar os pontos desenhados
trail_points = []

MAX_DISTANCE = 50
tempo_inicial = time.time()


In [17]:
# Caminho do vídeo de entrada caso use vídeo local



#video_path = 'Hands.mp4' #vídeo
#output_path = 'HandsResultado.mp4'
# Verifica se o vídeo existe
#if not os.path.isfile(video_path):
    #raise FileNotFoundError(f"Arquivo de vídeo '{video_path}' não encontrado.")

In [47]:
# Configura o vídeo de saída
cap = cv2.VideoCapture(0) #start the camera
fourcc = cv2.VideoWriter_fourcc(*'mp4v') 
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)) 
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)) 
frame_rate = int(cap.get(cv2.CAP_PROP_FPS)) 


#Caso queria gravação da saída
#out = cv2.VideoWriter(output_path, fourcc, frame_rate, (frame_width, frame_height))

In [48]:
pause_circles = [] #lista para as pausas
pause_duration = [] #lista para o tempo das pausas
pause_time_init = time.time() #contador para controlar o tempo da pausa
pause = False; #Variável para controlar entrada e sáida da pausa

strokes_euclidians = [] #Guarda as distâncias euclidianas ponto a cada ponto em um traço
strokes_length = []  #Guarda a distância total percorrida por cada traço
stroke = True; #Variável para controlar entrada e sáida de um novo traço

with mp_hands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.5) as hands:
    while cap.isOpened():#enquanto a camera estiver aberta
        time_frame = time.time() #início do contador do frame
        ret, frame = cap.read() #resultados da captura da camera
        if ret==False:
            print('fim')
            break
        frame = cv2.resize(frame, (frame_width, frame_height))
        
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)#OpenCV usa o formato de cores BGR, mas o MediaPipe usa RGB
        image.flags.writeable = False #setFlag

        #Detections
        results = hands.process(image)#Envia a imagem para o modelo do MediaPipe para detectar mãos.
        
        image.flags.writeable = True
        
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)#Retornando a imagem para BGR


        
        if time.time() - tempo_inicial > 20: # Contador usado para apagar os desenhos durante os testes
            trail_points.clear()  # Apaga o desenho
            tempo_inicial = time.time()  # Reinicia o contador

        if results.multi_hand_landmarks:

            #Desenha os pontos da mão identificados pelo modelo do MediaPipe
            for num, hand in enumerate(results.multi_hand_landmarks):
                mp_drawing.draw_landmarks(image, hand,mp_hands.HAND_CONNECTIONS,
                                        mp_drawing.DrawingSpec(color=(0, 250, 0), thickness=2, circle_radius=3),
                                        mp_drawing.DrawingSpec(color=(0, 0, 250), thickness=2, circle_radius=1),
                                        )

            #Mede a distancia do dedo indicador e polegar e guarda os pontos onde estavam próximos
            for hand_landmarks in results.multi_hand_landmarks:
                h, w, _ = frame.shape  # Altura e largura do frame

                # Coordenadas do polegar (landmark 4)
                x4 = int(hand_landmarks.landmark[4].x * w)
                y4 = int(hand_landmarks.landmark[4].y * h)

                # Coordenadas do indicador (landmark 8)
                x8 = int(hand_landmarks.landmark[8].x * w)
                y8 = int(hand_landmarks.landmark[8].y * h)
                
                distance = math.sqrt((x8 - x4) ** 2 + (y8 - y4) ** 2)
                # Se os pontos estiverem próximos, desenha no meio
                if distance < MAX_DISTANCE:
                    x_middle = (x4 + x8) // 2
                    y_middle = (y4 + y8) // 2

                    # Adiciona o ponto à trilha
                    trail_points.append((x_middle, y_middle))
                    

                # Desenha a linha entre os pontos salvos na trilha de pontos
                for i in range(1, len(trail_points)):
                    cv2.line(image, trail_points[i - 1], trail_points[i], (0, 255, 0), 3)
                    if (stroke == False):#Calculando euclidiana entre os pontos até que ocorra uma pausa
                        strokes_euclidians.append(math.sqrt((trail_points[i][0] - trail_points[i-1][0])**2 + (trail_points[i][1] - trail_points[i-1][1])**2))
                if (len(strokes_euclidians) != 0):
                    #Armazenando a distância do traço

                    print(f"Tamanho do traço: {sum(strokes_euclidians)}")
                    strokes_length.append(sum(strokes_euclidians))
                    strokes_euclidians.clear()
                    trail_points.clear()
                    stroke = True
                    
                if len(trail_points) > 7: 
                    dx = abs(trail_points[i][0] - trail_points[i - 3][0]) #Diferença de x
                    dy = abs(trail_points[i][1] - trail_points[i - 3][1]) #Diferença de y
                    if dx < 4 and  dy < 4 and pause == False: # Diferença mínima para 
                        print(f"Começo: dx:{dx} - dy:{dy}")
                        pause_circles.append(trail_points[i])
                        pause = True
                        pause_time_init = time.time()
                        strokes_euclidians.clear()
                        stroke = False
                    elif dx > 5 and dy > 5 and pause == True:
                        print(f"Saída: dx:{dx} - dy:{dy}")
                        print("_____________________________________")
                        pause = False
                        pause_duration.append(time.time() - pause_time_init)
                    
                # Desenha um círculo no último ponto detectado
                if len(trail_points) > 0:
                    cv2.circle(image, trail_points[-1], 5, (0, 0, 255), -1)
                # Desenha um círculo nos pontos de pausa                   
                for point in pause_circles:
                    cv2.circle(image, point, 5, (255, 0, 255), -1)

        
        #out.write(image) #Vídeo Saída
        cv2.imshow('Hand Tracking', image)# Mostrando a imagem
        if cv2.waitKey(10) & 0xFF == ord('q'):# Apertar q para parar
            break
        #times.append(time.time()-time_frame)
#print(sum(times)/len(times)) # Média de tempo de análise dos frames
cap.release()
#out.release() #Vídeo Saída
cv2.destroyAllWindows()
print('*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*')
print(f'Número de Pausas: {len(pause_circles)}')
print(f'Média de tempo das Pausas: {sum(pause_duration)/len(pause_duration)}')
print(f'Número de Traços: {len(strokes_length)}')
print(f'Média de tamanho dos Traços: {sum(strokes_length)/len(strokes_length)}')
#print(f"Vídeo com rastreamento salvo em: {output_path}")

Começo: dx:0 - dy:0
Tamanho do traço: 25.363070516249607
Saída: dx:21 - dy:6
_____________________________________
Começo: dx:0 - dy:0
Tamanho do traço: 777.715667930245
Saída: dx:443 - dy:32
_____________________________________
Começo: dx:3 - dy:1
Tamanho do traço: 126.55894703959541
Saída: dx:33 - dy:8
_____________________________________
Começo: dx:1 - dy:3
Tamanho do traço: 652.1993086941374
Saída: dx:359 - dy:60
_____________________________________
Começo: dx:3 - dy:0
Tamanho do traço: 879.3652713767258
Saída: dx:55 - dy:8
_____________________________________
Começo: dx:0 - dy:1
Tamanho do traço: 34.22072494775699
Saída: dx:393 - dy:9
_____________________________________
Começo: dx:0 - dy:3
Tamanho do traço: 623.8861228037518
Saída: dx:54 - dy:37
_____________________________________
Começo: dx:0 - dy:3
Tamanho do traço: 103.72508607440966
Saída: dx:38 - dy:11
_____________________________________
Começo: dx:1 - dy:2
Tamanho do traço: 1172.1109591126788
Saída: dx:39 - dy:8
__

In [4]:
mp_drawing.DrawingSpec??

Init signature:
mp_drawing.DrawingSpec(
    color: Tuple[int, int, int] = (224, 224, 224),
    thickness: int = 2,
    circle_radius: int = 2,
) -> None
Docstring:      DrawingSpec(color: Tuple[int, int, int] = (224, 224, 224), thickness: int = 2, circle_radius: int = 2)
Source:        
@dataclasses.dataclass
class DrawingSpec:
  # Color for drawing the annotation. Default to the white color.
  color: Tuple[int, int, int] = WHITE_COLOR
  # Thickness for drawing the annotation. Default to 2 pixels.
  thickness: int = 2
  # Circle radius. Default to 2 pixels.
  circle_radius: int = 2
File:           e:\code\anaconda\envs\mediapipe_env\lib\site-packages\mediapipe\python\solutions\drawing_utils.py
Type:           type
Subclasses:     

In [26]:
mp_hands.HAND_CONNECTIONS

frozenset({(0, 1),
           (0, 5),
           (0, 17),
           (1, 2),
           (2, 3),
           (3, 4),
           (5, 6),
           (5, 9),
           (6, 7),
           (7, 8),
           (9, 10),
           (9, 13),
           (10, 11),
           (11, 12),
           (13, 14),
           (13, 17),
           (14, 15),
           (15, 16),
           (17, 18),
           (18, 19),
           (19, 20)})